<a href="https://colab.research.google.com/github/emshashank/emshashank/blob/main/Presidio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install presidio_analyzer
!pip install presidio_anonymizer
!python -m spacy download en_core_web_lg
!pip install pdfminer.six
!pip install pikepdf
!pip install PyMuPDF reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [79]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00


In [2]:
# For Presidio
from presidio_analyzer import AnalyzerEngine, PatternRecognizer
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig

# For console output
from pprint import pprint

# For extracting text
from pdfminer.high_level import extract_text, extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextLine

# For updating the PDF
from pikepdf import Pdf, AttachedFileSpec, Name, Dictionary, Array

In [78]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
import fitz  # PyMuPDF

# Initialize engines
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

# Open the PDF
doc = fitz.open("./sample_data/pii/health_report-1.pdf")

# Store anonymized text for each page
anonymized_pages = []

for page in doc:
    text = page.get_text()

    # Analyze and anonymize PII
    results = analyzer.analyze(text=text, language='en')
    anonymized_text = anonymizer.anonymize(text=text, analyzer_results=results).text

    anonymized_pages.append(anonymized_text)

print(anonymized_pages)

[" \n \n \n \n \nName: DOE, <PERSON>: PM-XX-12345 \n \nDOB: <DATE_TIME> \n                    MRN: <UK_NHS> \n \n \nSpecimen: Blood, <IN_PAN> \nSex: Male \n                    Referring facility: Double Helix Hospital   \nLab Control Number: ABC123 \nRace/Ethnicity: White                           Referring physician: Dr. DNA \n \n \nReceived: <DATE_TIME> \nFamily #: <US_DRIVER_LICENSE>   \n                     Copies to: CGC \n \n \nPage: 1 of 4 \n \nTest(s) performed: Whole genome <IN_PAN> for test: Clinical diagnosis and family history of DCM with <IN_PAN>   \n \n \n \n \n \n \nAPPROACH \n<IN_PAN> of this <IN_PAN>'s genome was performed and the data was analyzed to identify <IN_PAN> reported and novel variants in (1) \n335 genes that have been <IN_PAN> in various cardiac diseases and <IN_PAN> (see <IN_PAN>  for a list of genes and coverage \ninformation); and (2) variants <IN_PAN> as disease-causing in public databases that have a minor allele frequency <5% in <NRP> or <NRP> chromos

In [80]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from PyPDF2 import PdfReader, PdfWriter
from io import BytesIO

# Step 1: Create overlay PDFs with anonymized text
overlay_pages = []

for text in anonymized_pages:
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=letter)
    can.setFont("Helvetica", 10)
    can.drawString(50, 750, text[:1000])  # Simplified: just first 1000 chars
    can.save()
    packet.seek(0)
    overlay_pages.append(PdfReader(packet).pages[0])

# Step 2: Overlay on original PDF
original = PdfReader("./sample_data/pii/health_report-1.pdf")
writer = PdfWriter()

for i in range(len(original.pages)):
    page = original.pages[i]
    overlay = overlay_pages[i]
    page.merge_page(overlay)  # Overlay anonymized text on top
    writer.add_page(page)

# Step 3: Save the anonymized PDF
with open("./sample_data/pii/health_report-1_anonymized.pdf", "wb") as f:
    writer.write(f)


In [92]:
import fitz  # PyMuPDF
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from collections import defaultdict

# Step 1: Initialize Presidio
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

# Step 2: Open original PDF
original_doc = fitz.open("./sample_data/pii/health_report-1.pdf")

# Step 3: Create a new blank PDF for anonymized output
anonymized_doc = fitz.open()

for orig_page in original_doc:
    # Extract all words: (x0, y0, x1, y1, word, block, line, word_no)
    words = orig_page.get_text("words")

    # Group words by line using Y-coordinates
    lines = defaultdict(list)
    for w in words:
        x0, y0, x1, y1, text, _, line_no, _ = w
        lines[line_no].append((x0, y0, x1, y1, text))

    # Create new blank page
    new_page = anonymized_doc.new_page(width=orig_page.rect.width, height=orig_page.rect.height)

    for line_no in sorted(lines.keys()):
        line = sorted(lines[line_no], key=lambda w: w[0])  # Sort words left to right
        reconstructed_line = ""
        positions = []
        last_x1 = None

        for word in line:
            x0, y0, x1, y1, text = word

            # Anonymize only if PERSON or DATE_TIME
            results = analyzer.analyze(text=text, language='en')
            filtered = [r for r in results if r.entity_type in ['PERSON', 'DATE_TIME']]
            if filtered:
                text = anonymizer.anonymize(text, analyzer_results=filtered).text

            # Estimate space by x-distance between words
            if last_x1 is not None and (x0 - last_x1) > 3:
                reconstructed_line += " "  # insert space if words are apart
            reconstructed_line += text
            last_x1 = x1
            positions.append((x0, y1))  # Use last position for line

        # Insert full line at position of first word
        if positions:
            pos = fitz.Point(positions[0][0], positions[0][1] - 1)
            new_page.insert_text(pos, reconstructed_line, fontsize=10, fontname="helv", color=(0, 0, 0))

# Step 4: Save the new anonymized file
anonymized_doc.save("./sample_data/pii/health_report-1_output.pdf")
anonymized_doc.close()
original_doc.close()


[(36.0, 87.25502014160156, 79.68000793457031, 107.44502258300781, 'Name:'), (83.03999328613281, 87.25502014160156, 116.15999603271484, 107.44502258300781, 'DOE,'), (119.51998138427734, 87.25502014160156, 156.10498046875, 107.44502258300781, '<PERSON>'), (360.0, 87.25502014160156, 427.8599548339844, 107.44502258300781, 'Accession'), (431.15997314453125, 87.25502014160156, 451.1999816894531, 107.44502258300781, 'ID:'), (454.55999755859375, 87.25502014160156, 549.3599853515625, 107.44502258300781, 'PM-XX-12345'), (36.0, 117.1346206665039, 59.19684600830078, 130.54078674316406, 'DOB:'), (61.43783950805664, 117.1346206665039, 118.92697143554688, 130.54078674316406, '<DATE_TIME>'), (197.05319213867188, 117.1346206665039, 221.6842803955078, 130.54078674316406, 'MRN:'), (223.81570434570312, 117.1346206665039, 283.0777282714844, 130.54078674316406, '<DATE_TIME>'), (396.00421142578125, 117.1346206665039, 443.1748046875, 130.54078674316406, 'Specimen:'), (445.4456481933594, 117.1346206665039, 474

In [75]:
# analyzer = AnalyzerEngine()
# anonymizer = AnonymizerEngine()

# analyzed_character_sets = []
# anonymized_character_sets = []
# anonymized_pages = []

# for page_layout in extract_pages("./sample_data/pii/health_report-1.pdf"):
#     for text_container in page_layout:
#         if isinstance(text_container, LTTextContainer):

#             # The element is a LTTextContainer, containing a paragraph of text.
#             text_to_anonymize = text_container.get_text()

#             # Analyze the text using the analyzer engine
#             analyzer_results = analyzer.analyze(text=text_to_anonymize, language='en')

#             anonymized_results = anonymizer.anonymize(
#                 text=text_to_anonymize,
#                 analyzer_results=analyzer_results,
#                 operators={"DEFAULT": OperatorConfig("replace", {"new_value": "<ANONYMIZED>"}),
#                                     "PHONE_NUMBER": OperatorConfig("mask", {"type": "mask", "masking_char" : "*", "chars_to_mask" : 12, "from_end" : True}),
#                                     "TITLE": OperatorConfig("redact", {})}
#             )
#             #print(anonymized_results)
#             if text_to_anonymize.isspace() == False:
#                 a=2
#                 #print(text_to_anonymize)
#                 #print(analyzer_results)

#             characters = list([])

#             # Grab the characters from the PDF
#             for text_line in filter(lambda t: isinstance(t, LTTextLine), text_container):
#                     for character in filter(lambda t: isinstance(t, LTChar), text_line):
#                             characters.append(character)

#             for result1 in anonymized_results.items:
#                 start = result.start
#                 end = result.end
#                 anonymized_character_sets.append({"characters": characters[start:end], "result": result1})


#             # Slice out the characters that match the analyzer results.
#             for result in analyzer_results:
#                 start = result.start
#                 end = result.end
#                 analyzed_character_sets.append({"characters": characters[start:end], "result": result})


# print(analyzed_character_sets)

[{'characters': [<LTChar 119.520,684.555,124.635,699.555 matrix=[15.00,0.00,0.00,15.00, (119.52,689.04)] font='KZWYZZ+Cambria-Bold' adv=0.341 text='J'>, <LTChar 124.680,684.555,135.105,699.555 matrix=[15.00,0.00,0.00,15.00, (124.68,689.04)] font='KZWYZZ+Cambria-Bold' adv=0.6950000000000001 text='O'>, <LTChar 135.120,684.555,145.950,699.555 matrix=[15.00,0.00,0.00,15.00, (135.12,689.04)] font='KZWYZZ+Cambria-Bold' adv=0.722 text='H'>, <LTChar 145.920,684.555,156.105,699.555 matrix=[15.00,0.00,0.00,15.00, (145.92,689.04)] font='KZWYZZ+Cambria-Bold' adv=0.679 text='N'>, <LTChar 156.000,684.555,159.300,699.555 matrix=[15.00,0.00,0.00,15.00, (156.00,689.04)] font='KZWYZZ+Cambria-Bold' adv=0.22 text=' '>, <LTChar 159.390,684.555,162.690,699.555 matrix=[15.00,0.00,0.00,15.00, (159.39,689.04)] font='KZWYZZ+Cambria-Bold' adv=0.22 text=' '>], 'result': type: PERSON, start: 11, end: 18, score: 0.85}, {'characters': [<LTChar 377.397,709.989,382.746,719.949 matrix=[9.96,0.00,0.00,9.96, (377.40,734.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.2 MB/s eta 0:00:00


In [10]:
# Combine the bounding boxes into a single bounding box.
def combine_rect(rectA, rectB):
    a, b = rectA, rectB
    startX = min( a[0], b[0] )
    startY = min( a[1], b[1] )
    endX = max( a[2], b[2] )
    endY = max( a[3], b[3] )
    return (startX, startY, endX, endY)

analyzed_bounding_boxes = []

# For each character set, combine the bounding boxes into a single bounding box.
for analyzed_character_set in analyzed_character_sets:
    completeBoundingBox = analyzed_character_set["characters"][0].bbox

    for character in analyzed_character_set["characters"]:
        completeBoundingBox = combine_rect(completeBoundingBox, character.bbox)

    analyzed_bounding_boxes.append({"boundingBox": completeBoundingBox, "result": analyzed_character_set["result"]})

In [72]:
# prompt: can you use the variable from above cell anonymized_results and use it to replace the data in pdf and write the anonymized data to pdf

from presidio_analyzer import AnalyzerEngine, PatternRecognizer
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig
from pprint import pprint
from pdfminer.high_level import extract_text, extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextLine, LTTextLineHorizontal
from pikepdf import Pdf, AttachedFileSpec, Name, Dictionary, Array

# ... (previous code)

# Updated code to replace text in the PDF
pdf = Pdf.open("./sample_data/pii/health_report-1.pdf")

for page in extract_pages("./sample_data/pii/health_report-1.pdf"):
      # Find the corresponding anonymized text
      anonymized_text = ""
      for text_container in page:
        if isinstance(text_container, LTTextContainer):

            # The element is a LTTextContainer, containing a paragraph of text.
            text_to_anonymize = text_container.get_text()
            #print(text_to_anonymize)
            analyzer_results = analyzer.analyze(text=text_to_anonymize, language='en')
            anonymized_results = anonymizer.anonymize(
              text=text_to_anonymize,
              analyzer_results=analyzer_results,
              operators={"DEFAULT": OperatorConfig("replace", {"new_value": "<ANONYMIZED>"}),
                "PHONE_NUMBER": OperatorConfig("mask", {"type": "mask", "masking_char" : "*", "chars_to_mask" : 12, "from_end" : True}),
                "TITLE": OperatorConfig("redact", {})}
            )
            #print(anonymized_results.text)


            for item in anonymized_results.items:
              if item.entity_type == "PERSON" :
                anonymized_text = item.text
                #print(item)
                break



pdf = Pdf.open("./sample_data/pii/health_report-1.pdf")

for page_num, page in enumerate(pdf.pages):
    for obj in page.get('/Contents'):
        print(obj.get_text())
        if isinstance(obj, Dictionary) and "/Rect" in obj:
          obj_bbox = obj["/Rect"]
          for analyzed_bounding_box in analyzed_bounding_boxes:
              bbox = analyzed_bounding_box["boundingBox"]
              anonymized_text = analyzed_bounding_box["result"].text
              if obj_bbox[0] <= bbox[0] and obj_bbox[1] <= bbox[1] and obj_bbox[2] >= bbox[2] and obj_bbox[3] >= bbox[3]:
                  if "/Contents" in obj.keys():
                      #replace contents
                      obj["/Contents"] = anonymized_text
                  elif "/Tj" in obj.keys():
                      #replace Tj
                      obj["/Tj"] = anonymized_text
pdf.save("./sample_data/pii/health_report-1-anonymized.pdf")
print("Anonymized PDF saved successfully!")


AttributeError: get_text

In [ ]:
# prompt: generate code to replace text in existing pdf with anonymized text using pikepdf

# ... (previous code)

# Updated code to replace text in the PDF



In [ ]:
# # prompt: can you write the anonymized_results back to the existing pdf with replaced data

# from presidio_analyzer import AnalyzerEngine, PatternRecognizer
# from presidio_anonymizer import AnonymizerEngine
# from presidio_anonymizer.entities import OperatorConfig
# from pprint import pprint
# from pdfminer.high_level import extract_text, extract_pages
# from pdfminer.layout import LTTextContainer, LTChar, LTTextLine
# from pikepdf import Pdf, AttachedFileSpec, Name, Dictionary, Array

# # ... (Existing code remains the same up to the point of creating analyzed_bounding_boxes)

# # Updated code to replace text in the PDF
# pdf = Pdf.open("./sample_data/pii/health_report-1.pdf")

# for page_num, page in enumerate(pdf.pages):
#     for analyzed_bounding_box in analyzed_bounding_boxes:
#         result = analyzed_bounding_box["result"]
#         bbox = analyzed_bounding_box["boundingBox"]

#         # Find the corresponding anonymized text
#         anonymized_text = ""
#         for item in anonymized_results.items:
#             if item.entity_type == result.entity_type and item.start == result.start and item.end == result.end:
#                 anonymized_text = item.text
#                 break  # Exit the loop once the match is found

#         if anonymized_text:  # Check if anonymized text was found
#             # Create a new text object with the anonymized text
#             new_text = pikepdf.Text(anonymized_text, bbox=bbox)

#             # Replace the original text with the new text object
#             for obj in page.get_contents():
#                 if obj.get('/Subtype') == '/Form':
#                   obj.add_resource(new_text)
#                   break  # Exit the inner loop after modifying content

# pdf.save("./sample_data/pii/health_report-1-anonymized.pdf")
# print("Anonymized PDF saved successfully!")


In [13]:
# pdf = Pdf.open("./sample_data/pii/health_report-1.pdf")

# annotations = []

# # Create a highlight annotation for each bounding box.
# for analyzed_bounding_box in analyzed_bounding_boxes:

#     boundingBox = analyzed_bounding_box["boundingBox"]

#     # Create the annotation.
#     # We could also create a redaction annotation if the ongoing workflows supports them.
#     highlight = Dictionary(
#         Type=Name.Annot,
#         Subtype=Name.Highlight,
#         QuadPoints=[boundingBox[0], boundingBox[3],
#                     boundingBox[2], boundingBox[3],
#                     boundingBox[0], boundingBox[1],
#                     boundingBox[2], boundingBox[1]],
#         Rect=[boundingBox[0], boundingBox[1], boundingBox[2], boundingBox[3]],
#         C=[1, 0, 0],
#         CA=0.5,
#         T=analyzed_bounding_box["result"].entity_type,
#     )

#     annotations.append(highlight)

# # Add the annotations to the PDF.
# pdf.pages[0].Annots = pdf.make_indirect(annotations)

# # And save.
# pdf.save("./sample_data/pii/health_report-1_annotated.pdf")